# Deep Learning: Image Classification

People rarely train deep neural networks from scratch (with random initialization) these days. Instead, we often use some form of [transfer learning](http://cs231n.github.io/transfer-learning/), which uses a [pretrained network](https://keras.rstudio.com/articles/applications.html) to save training time and improve model prediction. In this assignment, we will explore and compare two widely-used transfer learning techniques: feature extraction and fine-tuning.


**Before You Start**

Please make sure you have [numpy](https://numpy.org/), [Keras](https://keras.io/) and one of its backend engines (e.g., [TensorFlow](https://www.tensorflow.org/install/)).


**Dataset and Task**

We will use the [Viziometrics dataset](https://canvas.uw.edu/files/59549223/download?download_frd=1) ([paper1](https://ieeexplore.ieee.org/abstract/document/7888968), [paper2](https://arxiv.org/abs/1908.07465)). The dataset contains 15k+ images from scientific publications that are hand-labeled as one of the five classes: equation, photo, scheme, table, and visualization. The train, validation, and test set has already been splitted for us. Our task is to train neural network classifiers to predict the five labels.


**How to Complete This Assignment and What to Turn In**

We have provided some starter code. Each task you need to complete is marked with a red school bag 🎒and is worth a few points. Each task will either ask you to print something or report certain metrics. **Please submit your completed notebook with the code and the required output of each task. You will not receive full points if either code or output is missing.**

👉 **_Please start early. It takes time to train these networks._**

👉 **_This is a freshly minted assignment. If you experience confusions or struggles, please let us (zqu@uw.edu, billhowe@uw.edu) know so we can help!_**



## 0. Load the Data

🎒<font color='red'>(1 point)</font> Put the dataset in the same folder as this notebook, then run the following starter code. Because our training set contains a lot of images, we use [`ImageDataGenerator.flow_from_directory()`](https://keras.io/preprocessing/image/) to load images in batches. Please refer to [Vijayabhaskar's tutorial](https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720) to understand the parameterization of the generators. This starter code should output something like:

```
Found xxx images belonging to 5 classes.
Found xxx images belonging to 5 classes.
Found xxx images belonging to 5 classes.
```

Please include this output in your submission.

In [1]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, decode_predictions
from keras import layers
from keras.models import Model, Sequential
from keras.callbacks.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2733 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1563 images belonging to 5 classes.


## 1. Feature Extraction (20 points)

We will use a pretrained network (ResNet50) to extract features from the images. Then we will train a not-so-deep neural network on the features (X) and the labels (y). Finally we will evaluate our network. Please read [Vikas' tutorial on feature extraction](https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/) for code examples on feature extraction; we will use the same pipeline here.

🎒<font color='red'>(1 point)</font> Run the following code to load a [ResNet50 network](https://keras.rstudio.com/reference/application_resnet50.html) pretrained on the [ImageNet](http://www.image-net.org/) dataset. Note that we say `include_top=False` to exclude the last (or top) fully-connected layer which is responsible for classifying the labels in ImageNet; we want to use the earlier layers to extract features from our own dataset. Use `model.summary()` to inspect the ResNet50 architecture. This function should print out all the layers - their names, types, output shapes etc. Please include the printout in your submission.

In [3]:
# download the pre-trained ResNet50 model and remove last layer (include_top)
resnet = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

# inspect the ResNet50 architecture
resnet.summary()

# observe that the last layer, conv5_block3_out (Activation), has output shape 7 * 7 * 2048
# let's save this dimension for later use
DIM = 7 * 7 * 2048

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

🎒<font color='red'>(2 points)</font> Use ResNet50 to extract features from the images. Print the shape of the extracted features.

👉 We use [`model.predict()`](https://keras.io/models/sequential/#predict_generator) to extract features. This function will return the output of the last layer of ResNet50, namely, conv5_block3_out (Activation). These are the features captured by the pretrained network.

👉 This step is slow when you use the full training set. Setting `verbose=1` can make a wait a bit easier.

In [4]:
# # use ResNet50 to extract features from the images
train_features = resnet.predict(train_generator, verbose=1) # takes ~4min on my laptop
val_features = resnet.predict(val_generator, verbose=1)
test_features = resnet.predict(test_generator, verbose=1)

print(train_features.shape, val_features.shape, test_features.shape)

1563/1563 [==============================] - 278s 178ms/step
(2733, 7, 7, 2048) (1571, 7, 7, 2048) (1563, 7, 7, 2048)


🎒<font color='red'>(1 point)</font> Observe that the features extracted by ResNet50 are 4D arrays. Following common practice, we want to reshape the features into 2D arrays, which will be the input X of our not-so-deep neural network classifier. Run the following code and print the shape of the 2D arrays.

In [5]:
# reshape the features to 2D arrays
train_X = train_features.reshape((-1, DIM))
val_X = val_features.reshape((-1, DIM))
test_X = test_features.reshape((-1, DIM))

print(train_X.shape, val_X.shape, test_X.shape)

(2733, 100352) (1571, 100352) (1563, 100352)


🎒<font color='red'>(2 points)</font> Read the class labels from the generators. Print `test_labels`

In [6]:
# # read the class labels from the generators
train_labels = train_generator.classes
val_labels = val_generator.classes
test_labels = test_generator.classes

print(test_labels)

[0 0 0 ... 4 4 4]


🎒<font color='red'>(2 points)</font> Observe that the labels are not one-hot encoded. We need to one-hot encode these labels and they will be the y that our model will predict.

In [7]:
# # get one-hot encoding of labels
def get_one_hot(labels, nb_classes):
    res = np.eye(nb_classes)[np.array(labels).reshape(-1)]
    return res.reshape(list(labels.shape)+[nb_classes])

# # use get_one_hot()
NUM_CLASSES = 5
train_y = get_one_hot(train_labels, NUM_CLASSES)
val_y = get_one_hot(val_labels, NUM_CLASSES)
test_y = get_one_hot(test_labels, NUM_CLASSES)

In [37]:
train_y[7][0]

1.0

🎒<font color='red'>(3 points)</font> Define neural network that contains a few layers. There is a three-layer example network below. Please modify this network - you can add layers, remove layers, or change some of the parameters. Please print out your model architecture using `model.summary()`.

In [43]:
# create our model: a not-so-deep neural network
model = Sequential()

# input layer takes arrays of shape (*, DIM)
# todo: please modify this network to define your own model
model.add(layers.Dense(500, activation = "relu", input_shape=(DIM,))) 
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(8, activation='sigmoid'))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation = "softmax"))

# print out network architecture
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 500)               50176500  
_________________________________________________________________
dropout_8 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 1002      
_________________________________________________________________
dense_22 (Dense)             (None, 8)                 24        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 9         
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 10        
Total params: 50,177,545
Trainable params: 50,177,545
Non-trainable params: 0
__________________________________________

🎒<font color='red'>(1 point)</font> Compile your model using [`model.compile()`](https://keras.io/models/model/#compile). Choose `sgd` optimizer and `categorical_crossentropy` loss. Set `metrics = ["accuracy"]`

In [44]:
# # you need to compile the model before you can train it
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

🎒<font color='red'>(5 points)</font> Train the model you defined for about 10 epochs, or until the validation accuracy reaches 0.8 (whichever comes earlier). The model should be trained on `train_X` and `train_y`; it should be validated on `val_X` and `val_y`. Observe how the train accuracy and validation accuracy changes over epochs. Report (in a code comment) the best train and validation accuracy you've seen during training.

👉 Note that you can [check-point](https://keras.io/callbacks/#modelcheckpoint) model weights during training. After training, you can load back the weights of your best model.

In [45]:
# # save model weights while model is under training
filepath=('./viziometrics/-{epoch:02d}-{val_loss:.2f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# # train the model for about 20 epochs
history = model.fit(train_X,
                    train_y,
                    epochs=10,
                    batch_size=32,
                    validation_data=(val_X,val_y),
                    callbacks=[checkpoint])

Train on 2733 samples, validate on 1571 samples
Epoch 1/10
2733/2733 [==============================] - 16s 6ms/step - loss: 1.5458 - accuracy: 0.4391 - val_loss: 1.5994 - val_accuracy: 0.2342

Epoch 00001: val_accuracy improved from -inf to 0.23425, saving model to ./viziometrics/-01-1.60.hdf5
Epoch 2/10
2733/2733 [==============================] - 16s 6ms/step - loss: 1.4397 - accuracy: 0.5957 - val_loss: 1.6118 - val_accuracy: 0.3552

Epoch 00002: val_accuracy improved from 0.23425 to 0.35519, saving model to ./viziometrics/-02-1.61.hdf5
Epoch 3/10
2733/2733 [==============================] - 17s 6ms/step - loss: 1.3647 - accuracy: 0.6612 - val_loss: 1.6237 - val_accuracy: 0.3558

Epoch 00003: val_accuracy improved from 0.35519 to 0.35582, saving model to ./viziometrics/-03-1.62.hdf5
Epoch 4/10
2733/2733 [==============================] - 17s 6ms/step - loss: 1.2986 - accuracy: 0.6656 - val_loss: 1.6339 - val_accuracy: 0.3565

Epoch 00004: val_accuracy improved from 0.35582 to 0.356

🎒<font color='red'>(2 points)</font> Using your best model, report the test accuracy.

In [46]:
from keras.models import load_model

saved_model = load_model('./viziometrics/-10-1.67.hdf5')
# evaluate the model
_, train_acc = saved_model.evaluate(train_X, train_y, verbose=0)
_, test_acc = saved_model.evaluate(test_X, test_y, verbose=0)
print('Train Accuracy: %.3f, Test Accuracy: %.3f' % (train_acc, test_acc))

Train Accuracy: 0.676, Test Accuracy: 0.355


## 2. Fine-tuning (10 points)

Now we will explore the second technique: fine-tuning. We will append our own classification layers to the ResNet50 layers and train the entire network (from images to labels). We will start over from the generators.

In [47]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2733 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1563 images belonging to 5 classes.


🎒<font color='red'>(3 points)</font> Append classification layers to ResNet50. Please modify the example layers below to define your own classification layers. Print the network architecture of your own model.

In [52]:
# # add custom prediction layers to ResNet50
x = resnet.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)

predictions = layers.Dense(NUM_CLASSES, activation= 'softmax')(x)
model = Model(layers.Dense(10, activation='relu'))
model = Model(layers.Dense(10, activation='relu'))
model = Model(layers.Dense(10, activation='relu'))
model = Model(inputs = resnet.input, outputs = predictions)
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

🎒<font color='red'>(1 point)</font> Compile your model using [`model.compile()`](https://keras.io/models/model/#compile). Choose `sgd` optimizer and `categorical_crossentropy` loss. Set `metrics = ["accuracy"]`

In [53]:
l# # need to compile the model before training
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

🎒<font color='red'>(5 points)</font> Train the model you defined for about 10 epochs, or until the validation accuracy reaches 0.8 (whichever comes earlier). Checkpoint model weights during training. Report (in a code comment) the best train and validation accuracy you've seen during training.

In [54]:
# # train the model for about 10 epochs
STEP_SIZE_TRAIN=train_generator.n/train_generator.batch_size
STEP_SIZE_VAL=val_generator.n/val_generator.batch_size

filepath=('./viziolmetrics/-{epoch:02d}-{val_loss:.2f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VAL,
                    epochs=10,
                    callbacks=[checkpoint]
)

Epoch 1/10
86/85 [==============================] - 1777s 21s/step - loss: 0.2975 - accuracy: 0.9074 - val_loss: 1.0705 - val_accuracy: 0.7072

Epoch 00001: val_accuracy improved from -inf to 0.70719, saving model to ./viziometrics/-01-1.07.hdf5
Epoch 2/10
86/85 [==============================] - 1808s 21s/step - loss: 0.0837 - accuracy: 0.9755 - val_loss: 0.3797 - val_accuracy: 0.6747

Epoch 00002: val_accuracy did not improve from 0.70719
Epoch 3/10
86/85 [==============================] - 1760s 20s/step - loss: 0.0368 - accuracy: 0.9890 - val_loss: 1.3213 - val_accuracy: 0.8358

Epoch 00003: val_accuracy improved from 0.70719 to 0.83577, saving model to ./viziometrics/-03-1.32.hdf5
Epoch 4/10
86/85 [==============================] - 19251s 224s/step - loss: 0.0223 - accuracy: 0.9949 - val_loss: 2.5559 - val_accuracy: 0.8167

Epoch 00004: val_accuracy did not improve from 0.83577
Epoch 5/10
86/85 [==============================] - 1716s 20s/step - loss: 0.0154 - accuracy: 0.9974 - va

🎒<font color='red'>(1 point)</font> Using your best model, report the test accuracy.

In [55]:
# # measure test accuracy
from keras.models import load_model
saved_model = load_model('./viziometrics/-07-0.00.hdf5')
STEP_SIZE_VAL=val_generator.n/val_generator.batch_size
_,acc=saved_model.evaluate_generator(generator=val_generator,steps=STEP_SIZE_VAL)
print('Accuracy: %.3f' % (acc))

Accuracy: 0.852
